In [ ]:
# This code is compatible with python 3.6
!pip install numpy
!pip install XlsxWriter
!pip install pandas
!pip install coordinates-converter

In [ ]:
import numpy as np #for numerical computations
import pandas as pd #for reading the dataset
import time #calculating the execution time
import xlsxwriter #for writing the data to a new excel file
import decimal #for numerical computation
from coordinates.converter import CoordinateConverter, WGS84, L_Est97
converter = CoordinateConverter

In [ ]:
######################-----------This code is for extracting the entire trajectory data--------#################


start_time = time.time()

dataset = pd.read_csv('competition_dataset.csv') # Reads the competition dataset

column_names = dataset.columns[0] 
column_names = column_names.split(";") # The column names are seperated

workbook1 = xlsxwriter.Workbook('processed_full1.xlsx') # New excel file is created where data will be stored
worksheet1 = workbook1.add_worksheet() # Adding a worksheet to the excel file

for i in range(10):
    if i <= 10:
        worksheet1.write(0,i,column_names[i]) #Writing the column names to the first row

worksheet1.write(0,10,'modified_lat')
worksheet1.write(0,11,'modified_long')
counter = 0 #Determines the row in which data has to be written
sheet_no = 0 # Sheet No for current data writing (Because entire data cannot be accomodated to a single sheet)
no_vehicles_in_one_sheet = 300 #(Because entire data cannot be accomodated to a single sheet)
no_of_columns_repeated = 6 #Last six columns are repeated in the given data

for i in range(len(dataset)):# Iterate through the different rows of vehicles
    counter = counter + 1
    
    if int(i/no_vehicles_in_one_sheet) > sheet_no: # This condition is used to add new worksheet when "no_vehicles_in_one_sheet" are added
        worksheet1 = workbook1.add_worksheet()
        sheet_no = int(i/no_vehicles_in_one_sheet)
        counter = 1
        for m in range(10):
            worksheet1.write(0,m,column_names[m]) #Writing the column names to the first row of the new sheet
        worksheet1.write(0,10,'modified_lat')
        worksheet1.write(0,11,'modified_long')
    
    row_details = dataset.iloc[i,0].split(";") # Extracts the details of one vehicle seperated by ";" into a list
    for j in range(10): # This for loop writes the first ten columns of the vehicle details
        if j==1:
            worksheet1.write(counter, j, row_details[j]) # This is the name of the vehicle so its data type must be string
        elif j==4:
            wgs_point = WGS84(lat=float(row_details[j]), long=float(row_details[j+1]))
            temp1 = converter.wgs84_to_l_est97(wgs_point)
            worksheet1.write(counter, j, float(row_details[j]))
            worksheet1.write(counter, 10, temp1[0])
        elif j==5:
            worksheet1.write(counter, j, float(row_details[j]))
            worksheet1.write(counter, 11, temp1[0])
        else:
            worksheet1.write(counter, j, float(row_details[j])) # All other details of vehicles should have data type float
            
    temp = -1 # This determines whether the six details of the vehicles have been written or not. If yes, the counter is incremented
    
    for j in range(10,len(row_details)-1): # In row_details, last element of every list is blank character. Therefore it is ignored.
        
        if int((j-10)/no_of_columns_repeated) > temp: # This determines whether the six details of the vehicles have been written or not. If yes, the counter is incremented
            temp = int((j-10)/no_of_columns_repeated)
            counter = counter + 1
            
        for k in range(4):
            if k==1:
                worksheet1.write(counter, k, row_details[k])
            else:
                worksheet1.write(counter, k, float(row_details[k]))
        
        t = (j-10)%no_of_columns_repeated + 4 # This gives the appropriate column in which the data has to be written
        
        if t==4:
            wgs_point = WGS84(lat=float(row_details[j]), long=float(row_details[j+1]))
            temp1 = converter.wgs84_to_l_est97(wgs_point)
            worksheet1.write(counter, 10, temp1[0])
            worksheet1.write(counter, t, float(row_details[j]))
        elif t==5:
            worksheet1.write(counter, t, float(row_details[j]))
            worksheet1.write(counter, 11, temp1[1])
        else:
            worksheet1.write(counter, t, float(row_details[j])) # All other details of vehicles should have data type float

workbook1.close()
print('Time required for code in processing the data [s]: ', time.time() - start_time) # Prints the time required for preparing the trajectory data